<div class="alert alert-block alert-danger">

# **Performance of the Eversense versus the Free Style Libre Flash glucose monitor during exercise and normal daily activities in subjects with type 1 diabetes mellitus**
    
</div>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from collections import Counter
import scipy.stats as stats

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFE
from sklearn import metrics
from sklearn.metrics import r2_score, accuracy_score, confusion_matrix, roc_curve, f1_score
import statsmodels.api as sm
from sklearn.svm import SVC
from sklearn import svm
from scipy.optimize import curve_fit
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy import stats
import seaborn as sn
# from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn import tree
# import pydotplus
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import LeaveOneOut
from scipy.stats import fisher_exact
from scipy.stats import chi2_contingency, mood, shapiro
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import mutual_info_classif, f_regression, mutual_info_regression, f_classif

from random import choices

import shap

In [ ]:
dfw1 = pd.read_excel("Week 1.xlsx")
dfw2 = pd.read_excel("Week 2.xlsx")

dfw1['Datum'] = dfw1['Datum'].astype(str)
dfw2['Datum'] = dfw2['Datum'].astype(str)

dfw1.rename(columns={"Fingerprick  (mg/dL)": "Fingerprick (mg/dL)"}, inplace = True)

feature = "Patiënt ID"
elset = list(set(dfw1[feature].tolist()))

dfw1_dict = {idd: dfw1.loc[dfw1['Patiënt ID'] == idd] for idd in elset}
dfw2_dict = {idd: dfw2.loc[dfw2['Patiënt ID'] == idd] for idd in elset}

In [ ]:
dfw1["Period"] = ['PA']*dfw1.shape[0]
dfw2["Period"] = ['NDA']*dfw2.shape[0]

dff = pd.concat([dfw1, dfw2], axis=0)

In [ ]:
dfw1['Datum'] = dfw1['Datum'].str[0:10]
dfw2['Datum'] = dfw2['Datum'].str[0:10]

dfw1_dict = {idd: dfw1.loc[dfw1['Patiënt ID'] == idd] for idd in elset}
dfw2_dict = {idd: dfw2.loc[dfw2['Patiënt ID'] == idd] for idd in elset}

In [ ]:
dfw1 = pd.read_excel("Week 1.xlsx")
dfw2 = pd.read_excel("Week 2.xlsx")

In [ ]:
dfw1['Datum'] = dfw1['Datum'].astype(str)
dfw2['Datum'] = dfw2['Datum'].astype(str)

dfw1.rename(columns={"Fingerprick  (mg/dL)": "Fingerprick (mg/dL)"}, inplace = True)

dfw1['Datum'] = dfw1['Datum'].str[11:13]
dfw2['Datum'] = dfw2['Datum'].str[11:13]
dfw1['Datum'] = dfw1['Datum'].astype(float)
dfw2['Datum'] = dfw2['Datum'].astype(float)

In [ ]:
nochew1 = dfw1.loc[(dfw1["Datum"] < 6)]
mañanaw1 = dfw1.loc[(dfw1["Datum"] >= 6) & (dfw1["Datum"] < 12)]
tardew1 = dfw1.loc[(dfw1["Datum"] >= 12) & (dfw1["Datum"] < 18)]
eveningw1 = dfw1.loc[(dfw1["Datum"] >= 18)]

In [ ]:
nochew2 = dfw2.loc[(dfw2["Datum"] < 6)]
mañanaw2 = dfw2.loc[(dfw2["Datum"] >= 6) & (dfw2["Datum"] < 12)]
tardew2 = dfw2.loc[(dfw2["Datum"] >= 12) & (dfw2["Datum"] < 18)]
eveningw2 = dfw2.loc[(dfw2["Datum"] >= 18)]

In [ ]:
def plot_custom_boxplot_with_ci(df_ci, metric,title_figure='mean_std_plot', type_variables='selected',flag_save_figure=False, flag_save_features=False):

    # df_ci = df_ci.sort_values('variable_name', ascending=False)

    fig, ax = plt.subplots(1, 1, figsize=(12, 6))

    v_mean = np.mean((df_ci['ci_upper'].values, df_ci['ci_lower'].values), axis=0)
    df_ci['mean'] = v_mean

    list_features_nonselected = []
    list_features_selected = []
    list_features_all = []

    for index, row in df_ci.iterrows():
        if min(row['ci_lower'], row['ci_upper']) <= 0.0 <= max(row['ci_lower'], row['ci_upper']):
            color_interval = 'red'
            legend_item = 'No differences'
            list_features_nonselected.append((row['var_name'], row['variable_name'], float(row['ci_lower']),
                                           float(row['ci_upper']), float(row['mean']), color_interval, legend_item))
        else:
            color_interval = 'blue'
            legend_item = 'Differences'
            list_features_selected.append((row['var_name'], row['variable_name'], float(row['ci_lower']),
                                       float(row['ci_upper']), float(row['mean']), color_interval, legend_item))

        list_features_all.append((row['var_name'], row['variable_name'], float(row['ci_lower']),
                                   float(row['ci_upper']), float(row['mean']), color_interval, legend_item))

    if type_variables == 'nonselected':
        list_features_filtered = list_features_nonselected
    elif type_variables == 'selected':
        list_features_filtered = list_features_selected
    else:
        list_features_filtered = list_features_all

    m_features_selected = np.array(list_features_filtered)
    df_features_selected = pd.DataFrame(m_features_selected, columns=['var_name', 'variable_name', 'ci_lower', 'ci_upper', 'mean', 'color_interval', 'legend_item'])

    df_features_selected['ci_lower'] = pd.to_numeric(df_features_selected['ci_lower'])
    df_features_selected['ci_upper'] = pd.to_numeric(df_features_selected['ci_upper'])
    df_features_selected['mean'] = pd.to_numeric(df_features_selected['mean'])

    for index, row in df_features_selected.iterrows():

        ax.plot(row['mean'], index, marker='o', ms=5, linewidth=1, ls='', color=row['color_interval'], label=row['legend_item'])
        ax.hlines(row['variable_name'], row['ci_lower'], row['ci_upper'], label='', lw=2,
                  color=row['color_interval'], ls='-')

        # ax.plot(row2['mean'], index2, marker='o', ms=5, linewidth=1, ls='', color=color_interval, label=legend_item)
        # ax.hlines(row2['var_name'], row2['ci_lower'], row2['ci_upper'], label='', lw=2, color=color_interval, ls='-')

    # fig, ax = plt.subplots(1, 1, figsize=(12, 14))
    # ax.plot(df_features_selected['mean'], df_features_selected['var_name'], marker='o', ms=5, linewidth=1, ls='',
    #         color=df_features_selected['color_interval'], label=df_features_selected['legend_item'])
    # ax.hlines(df_features_selected['var_name'], df_features_selected['ci_lower'], df_features_selected['ci_upper'],
    #           label='', lw=2, color='red', ls='-')

    #plt.axvline(x=0, color='k', linestyle='--')
    plt.yticks(fontsize=24, weight = 'bold')
    plt.xticks(fontsize=24, weight = 'bold')
    plt.xlim(-32, 32)
    # plt.yticks(list_features_selected)
    #plt.legend(fontsize=18)

    if flag_save_features:
        

        df_c = pd.DataFrame(np.array(list_features_selected))
        df_c.to_csv(str(Path.joinpath(consts.PATH_PROJECT_FS, 'df_coeff_fs.csv')))

    ax.grid(axis='x', ls='-', lw=1, alpha=0.5)
    ax.grid(which='major', color='#CCCCCC', linestyle='--')
    ax.grid(which='minor', color='#CCCCCC', linestyle=':')

    #legend_without_duplicate_labels(ax)

    plt.tick_params(
        axis='y',
        which='both',
        bottom=False,
        top=False,
        labelbottom=False
    )

    if flag_save_figure:
        name = 'bootstrap_KW_' + metric + '.pdf'
        plt.savefig(name, bbox_inches = 'tight')
#         fig.tight_layout()
#         fig.savefig(str(Path.joinpath(consts.PATH_PROJECT_FIGURES, '{}.pdf'.format(title_figure))))
#                     # dpi=h/magic_height*dpi)
    else:
        plt.show()

import matplotlib.font_manager as font_manager

In [ ]:
def legend_without_duplicate_labels(ax):
    font = font_manager.FontProperties(weight='bold',
                                   style='normal', size=24)
    handles, labels = ax.get_legend_handles_labels()
    unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    ax.legend(*zip(*unique),prop = font)

In [ ]:
orange = sns.color_palette()[0]
blue = sns.color_palette()[1]

<div class="alert alert-block alert-danger">

**MAD and MARD**
    
</div>

**Para sacar el MAD no incluir la division "/dfw1['Fingerprick (mg/dL)']"**

In [ ]:
device = 'Eversense (mg/dl)'

#Full day
mad_series_w1 = np.abs(dfw1['Fingerprick (mg/dL)'] - dfw1[device])/dfw1['Fingerprick (mg/dL)']
mad_series_w2 = np.abs(dfw2['Fingerprick (mg/dL)'] - dfw2[device])/dfw2['Fingerprick (mg/dL)']
shapiro(mad_series_w1)
shapiro(mad_series_w2)

print('PA MAD :', np.mean(mad_series_w1),' +-', np.std(mad_series_w1))
print('NDA MAD:', np.mean(mad_series_w2),' +-', np.std(mad_series_w2))
tval, pval = stats.mannwhitneyu(mad_series_w1, mad_series_w2)
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the distribution is different between PA and", device, 'NDA hypoglicemia')
else:
    print("The pvalue is: ", pval, " so we accept H0: the distribution is NOT different between PA and", device, 'NDA hypoglicemia')

print('---------------')

In [ ]:
device = 'FSL (mg/dL)'

#Full day
mad_series_w1 = np.abs(dfw1['Fingerprick (mg/dL)'] - dfw1[device])/dfw1['Fingerprick (mg/dL)']
mad_series_w2 = np.abs(dfw2['Fingerprick (mg/dL)'] - dfw2[device])/dfw2['Fingerprick (mg/dL)']
shapiro(mad_series_w1)
shapiro(mad_series_w2)

print('PA MAD :', np.mean(mad_series_w1),' +-', np.std(mad_series_w1))
print('NDA MAD:', np.mean(mad_series_w2),' +-', np.std(mad_series_w2))
tval, pval = stats.mannwhitneyu(mad_series_w1, mad_series_w2)
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the distribution is different between PA and", device, 'NDA hypoglicemia')
else:
    print("The pvalue is: ", pval, " so we accept H0: the distribution is NOT different between PA and", device, 'NDA hypoglicemia')

print('---------------')

<div class="alert alert-block alert-warning">
    
**MAD/MARD per time intervals**

</div>

In [ ]:
device = 'Eversense (mg/dl)'

In [ ]:
#M
mad_series_w1 = np.abs(mañanaw1['Fingerprick (mg/dL)'] - mañanaw1[device])/mañanaw1['Fingerprick (mg/dL)']
mad_series_w2 = np.abs(mañanaw2['Fingerprick (mg/dL)'] - mañanaw2[device])/mañanaw2['Fingerprick (mg/dL)']
shapiro(mad_series_w1)
shapiro(mad_series_w2)

print('PA MAD morn:', np.mean(mad_series_w1),' +-', np.std(mad_series_w1))
print('NDA MAD morn:', np.mean(mad_series_w2),' +-', np.std(mad_series_w2))
tval, pval = stats.mannwhitneyu(mad_series_w1, mad_series_w2)
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the distribution is different between PA and", device, 'NDA hypoglicemia')
else:
    print("The pvalue is: ", pval, " so we accept H0: the distribution is NOT different between PA and", device, 'NDA hypoglicemia')

print('---------------')

In [ ]:
#A
mad_series_w1 = np.abs(tardew1['Fingerprick (mg/dL)'] - tardew1[device])/tardew1['Fingerprick (mg/dL)']
mad_series_w2 = np.abs(tardew2['Fingerprick (mg/dL)'] - tardew2[device])/tardew2['Fingerprick (mg/dL)']
shapiro(mad_series_w1)
shapiro(mad_series_w2)

print('PA MAD aft:', np.mean(mad_series_w1),' +-', np.std(mad_series_w1))
print('NDA MAD aft:', np.mean(mad_series_w2),' +-', np.std(mad_series_w2))
tval, pval = stats.mannwhitneyu(mad_series_w1, mad_series_w2)
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the distribution is different between PA and", device, 'NDA hypoglicemia')
else:
    print("The pvalue is: ", pval, " so we accept H0: the distribution is NOT different between PA and", device, 'NDA hypoglicemia')

print('---------------')

In [ ]:
#E
mad_series_w1 = np.abs(eveningw1['Fingerprick (mg/dL)'] - eveningw1[device])/eveningw1['Fingerprick (mg/dL)']
mad_series_w2 = np.abs(eveningw2['Fingerprick (mg/dL)'] - eveningw2[device])/eveningw2['Fingerprick (mg/dL)']
shapiro(mad_series_w1)
shapiro(mad_series_w2)

print('PA MAD eve:', np.mean(mad_series_w1),' +-', np.std(mad_series_w1))
print('NDA MAD eve:', np.mean(mad_series_w2),' +-', np.std(mad_series_w2))
tval, pval = stats.mannwhitneyu(mad_series_w1, mad_series_w2)
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the distribution is different between PA and", device, 'NDA hypoglicemia')
else:
    print("The pvalue is: ", pval, " so we accept H0: the distribution is NOT different between PA and", device, 'NDA hypoglicemia')

print('---------------')

In [ ]:
#N
mad_series_w1 = np.abs(nochew1['Fingerprick (mg/dL)'] - nochew1[device])/nochew1['Fingerprick (mg/dL)']
mad_series_w2 = np.abs(nochew2['Fingerprick (mg/dL)'] - nochew2[device])/nochew2['Fingerprick (mg/dL)']
shapiro(mad_series_w1)
shapiro(mad_series_w2)

print('PA MAD nig:', np.mean(mad_series_w1),' +-', np.std(mad_series_w1))
print('NDA MAD nig:', np.mean(mad_series_w2),' +-', np.std(mad_series_w2))
tval, pval = stats.mannwhitneyu(mad_series_w1, mad_series_w2)
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the distribution is different between PA and", device, 'NDA hypoglicemia')
else:
    print("The pvalue is: ", pval, " so we accept H0: the distribution is NOT different between PA and", device, 'NDA hypoglicemia')

print('---------------')

In [ ]:
device = 'FSL (mg/dL)'

In [ ]:
#M
mad_series_w1 = np.abs(mañanaw1['Fingerprick (mg/dL)'] - mañanaw1[device])/mañanaw1['Fingerprick (mg/dL)']
mad_series_w2 = np.abs(mañanaw2['Fingerprick (mg/dL)'] - mañanaw2[device])/mañanaw2['Fingerprick (mg/dL)']
shapiro(mad_series_w1)
shapiro(mad_series_w2)

print('PA MAD morn:', np.mean(mad_series_w1),' +-', np.std(mad_series_w1))
print('NDA MAD morn:', np.mean(mad_series_w2),' +-', np.std(mad_series_w2))
tval, pval = stats.mannwhitneyu(mad_series_w1, mad_series_w2)
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the distribution is different between PA and", device, 'NDA hypoglicemia')
else:
    print("The pvalue is: ", pval, " so we accept H0: the distribution is NOT different between PA and", device, 'NDA hypoglicemia')

print('---------------')

In [ ]:
#A
mad_series_w1 = np.abs(tardew1['Fingerprick (mg/dL)'] - tardew1[device])/tardew1['Fingerprick (mg/dL)']
mad_series_w2 = np.abs(tardew2['Fingerprick (mg/dL)'] - tardew2[device])/tardew2['Fingerprick (mg/dL)']
shapiro(mad_series_w1)
shapiro(mad_series_w2)

print('PA MAD aft:', np.mean(mad_series_w1),' +-', np.std(mad_series_w1))
print('NDA MAD aft:', np.mean(mad_series_w2),' +-', np.std(mad_series_w2))
tval, pval = stats.mannwhitneyu(mad_series_w1, mad_series_w2)
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the distribution is different between PA and", device, 'NDA hypoglicemia')
else:
    print("The pvalue is: ", pval, " so we accept H0: the distribution is NOT different between PA and", device, 'NDA hypoglicemia')

print('---------------')

In [ ]:
#E
mad_series_w1 = np.abs(eveningw1['Fingerprick (mg/dL)'] - eveningw1[device])/eveningw1['Fingerprick (mg/dL)']
mad_series_w2 = np.abs(eveningw2['Fingerprick (mg/dL)'] - eveningw2[device])/eveningw2['Fingerprick (mg/dL)']
shapiro(mad_series_w1)
shapiro(mad_series_w2)

print('PA MAD eve:', np.mean(mad_series_w1),' +-', np.std(mad_series_w1))
print('NDA MAD eve:', np.mean(mad_series_w2),' +-', np.std(mad_series_w2))
tval, pval = stats.mannwhitneyu(mad_series_w1, mad_series_w2)
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the distribution is different between PA and", device, 'NDA hypoglicemia')
else:
    print("The pvalue is: ", pval, " so we accept H0: the distribution is NOT different between PA and", device, 'NDA hypoglicemia')

print('---------------')

In [ ]:
#N
mad_series_w1 = np.abs(nochew1['Fingerprick (mg/dL)'] - nochew1[device])/nochew1['Fingerprick (mg/dL)']
mad_series_w2 = np.abs(nochew2['Fingerprick (mg/dL)'] - nochew2[device])/nochew2['Fingerprick (mg/dL)']
shapiro(mad_series_w1)
shapiro(mad_series_w2)

print('PA MAD nig:', np.mean(mad_series_w1),' +-', np.std(mad_series_w1))
print('NDA MAD nig:', np.mean(mad_series_w2),' +-', np.std(mad_series_w2))
tval, pval = stats.mannwhitneyu(mad_series_w1, mad_series_w2)
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the distribution is different between PA and", device, 'NDA hypoglicemia')
else:
    print("The pvalue is: ", pval, " so we accept H0: the distribution is NOT different between PA and", device, 'NDA hypoglicemia')

print('---------------')

<div class="alert alert-block alert-warning">
    
**MAD/MARD per glucose ranges**

</div>

In [ ]:
#PA
hypo_ref_w1 = dfw1.loc[(dfw1['Fingerprick (mg/dL)'] < 70)]
normo_ref_w1 = dfw1.loc[(dfw1['Fingerprick (mg/dL)'] >= 70) & (dfw1['Fingerprick (mg/dL)'] < 180)]
hyper_ref_w1 = dfw1.loc[(dfw1['Fingerprick (mg/dL)'] >= 180)]

#NDA
hypo_ref_w2 = dfw2.loc[(dfw2['Fingerprick (mg/dL)'] < 70)]
normo_ref_w2 = dfw2.loc[(dfw2['Fingerprick (mg/dL)'] >= 70) & (dfw2['Fingerprick (mg/dL)'] < 180)]
hyper_ref_w2 = dfw2.loc[(dfw2['Fingerprick (mg/dL)'] >= 180)]

In [ ]:
device = 'Eversense (mg/dl)'

#hypo
mad_hypo_series_w1 = np.abs(hypo_ref_w1['Fingerprick (mg/dL)'] - hypo_ref_w1[device])#/hypo_ref_w1['Fingerprick (mg/dL)']
mad_hypo_series_w2 = np.abs(hypo_ref_w2['Fingerprick (mg/dL)'] - hypo_ref_w2[device])#/hypo_ref_w2['Fingerprick (mg/dL)']
shapiro(mad_hypo_series_w1)
shapiro(mad_hypo_series_w2)

print('PA MAD hypo:', np.mean(mad_hypo_series_w1),' +-', np.std(mad_hypo_series_w1))
print('NDA MAD hypo:', np.mean(mad_hypo_series_w2),' +-', np.std(mad_hypo_series_w2))
tval, pval = stats.mannwhitneyu(mad_hypo_series_w1, mad_hypo_series_w2)
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the distribution is different between PA and", device, 'NDA hypoglicemia')
else:
    print("The pvalue is: ", pval, " so we accept H0: the distribution is NOT different between PA and", device, 'NDA hypoglicemia')

print('---------------')

In [ ]:
#normo
mad_normo_series_w1 = np.abs(normo_ref_w1['Fingerprick (mg/dL)'] - normo_ref_w1[device])#/normo_ref_w1['Fingerprick (mg/dL)']
mad_normo_series_w2 = np.abs(normo_ref_w2['Fingerprick (mg/dL)'] - normo_ref_w2[device])#/normo_ref_w2['Fingerprick (mg/dL)']
shapiro(mad_normo_series_w1)
shapiro(mad_normo_series_w2)

print('PA MAD notmo:', np.mean(mad_normo_series_w1),' +-', np.std(mad_normo_series_w1))
print('NDA MAD normo:', np.mean(mad_normo_series_w2),' +-', np.std(mad_normo_series_w2))

tval, pval = stats.mannwhitneyu(mad_normo_series_w1, mad_normo_series_w2)
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the distribution is different between PA and", device, 'NDA hyperglicemia')
else:
    print("The pvalue is: ", pval, " so we accept H0: the distribution is NOT different between PA and", device, 'NDA hyperglicemia')        

print('---------------')  

In [ ]:
#hyper
mad_hyper_series_w1 = np.abs(hyper_ref_w1['Fingerprick (mg/dL)'] - hyper_ref_w1[device])#/hyper_ref_w1['Fingerprick (mg/dL)']
mad_hyper_series_w2 = np.abs(hyper_ref_w2['Fingerprick (mg/dL)'] - hyper_ref_w2[device])#/hyper_ref_w2['Fingerprick (mg/dL)']
shapiro(mad_hyper_series_w1)
shapiro(mad_hyper_series_w2)

print('PA MAD hyper:', np.mean(mad_hyper_series_w1),' +-', np.std(mad_hyper_series_w1))
print('NDA MAD hyper:', np.mean(mad_hyper_series_w2),' +-', np.std(mad_hyper_series_w2))

tval, pval = stats.mannwhitneyu(mad_hyper_series_w1, mad_hyper_series_w2)
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the distribution is different between PA and", device, 'NDA hyperglicemia')
else:
    print("The pvalue is: ", pval, " so we accept H0: the distribution is NOT different between PA and", device, 'NDA hyperglicemia')    

In [ ]:
#PA
hypo_ref_w1 = dfw1.loc[(dfw1['Fingerprick (mg/dL)'] < 70)]
normo_ref_w1 = dfw1.loc[(dfw1['Fingerprick (mg/dL)'] >= 70) & (dfw1['Fingerprick (mg/dL)'] < 180)]
hyper_ref_w1 = dfw1.loc[(dfw1['Fingerprick (mg/dL)'] >= 180)]

#NDA
hypo_ref_w2 = dfw2.loc[(dfw2['Fingerprick (mg/dL)'] < 70)]
normo_ref_w2 = dfw2.loc[(dfw2['Fingerprick (mg/dL)'] >= 70) & (dfw2['Fingerprick (mg/dL)'] < 180)]
hyper_ref_w2 = dfw2.loc[(dfw2['Fingerprick (mg/dL)'] >= 180)]

device = 'FSL (mg/dL)'

In [ ]:
#hypo
mad_hypo_series_w1 = np.abs(hypo_ref_w1['Fingerprick (mg/dL)'] - hypo_ref_w1[device])#/hypo_ref_w1['Fingerprick (mg/dL)']
mad_hypo_series_w2 = np.abs(hypo_ref_w2['Fingerprick (mg/dL)'] - hypo_ref_w2[device])#/hypo_ref_w2['Fingerprick (mg/dL)']
shapiro(mad_hypo_series_w1)
shapiro(mad_hypo_series_w2)

print('PA MAD hypo:', np.mean(mad_hypo_series_w1),' +-', np.std(mad_hypo_series_w1))
print('NDA MAD hypo:', np.mean(mad_hypo_series_w2),' +-', np.std(mad_hypo_series_w2))
tval, pval = stats.mannwhitneyu(mad_hypo_series_w1, mad_hypo_series_w2)
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the distribution is different between PA and", device, 'NDA hypoglicemia')
else:
    print("The pvalue is: ", pval, " so we accept H0: the distribution is NOT different between PA and", device, 'NDA hypoglicemia')

print('---------------')

In [ ]:
##normo
mad_normo_series_w1 = np.abs(normo_ref_w1['Fingerprick (mg/dL)'] - normo_ref_w1[device])#/normo_ref_w1['Fingerprick (mg/dL)']
mad_normo_series_w2 = np.abs(normo_ref_w2['Fingerprick (mg/dL)'] - normo_ref_w2[device])#/normo_ref_w2['Fingerprick (mg/dL)']
shapiro(mad_normo_series_w1)
shapiro(mad_normo_series_w2)

print('PA MAD notmo:', np.mean(mad_normo_series_w1),' +-', np.std(mad_normo_series_w1))
print('NDA MAD normo:', np.mean(mad_normo_series_w2),' +-', np.std(mad_normo_series_w2))

tval, pval = stats.mannwhitneyu(mad_normo_series_w1, mad_normo_series_w2)
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the distribution is different between PA and", device, 'NDA hyperglicemia')
else:
    print("The pvalue is: ", pval, " so we accept H0: the distribution is NOT different between PA and", device, 'NDA hyperglicemia')        

print('---------------')  

In [ ]:
#hyper
mad_hyper_series_w1 = np.abs(hyper_ref_w1['Fingerprick (mg/dL)'] - hyper_ref_w1[device])#/hyper_ref_w1['Fingerprick (mg/dL)']
mad_hyper_series_w2 = np.abs(hyper_ref_w2['Fingerprick (mg/dL)'] - hyper_ref_w2[device])#/hyper_ref_w2['Fingerprick (mg/dL)']
shapiro(mad_hyper_series_w1)
shapiro(mad_hyper_series_w2)

print('PA MAD hyper:', np.mean(mad_hyper_series_w1),' +-', np.std(mad_hyper_series_w1))
print('NDA MAD hyper:', np.mean(mad_hyper_series_w2),' +-', np.std(mad_hyper_series_w2))

tval, pval = stats.mannwhitneyu(mad_hyper_series_w1, mad_hyper_series_w2)
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the distribution is different between PA and", device, 'NDA hyperglicemia')
else:
    print("The pvalue is: ", pval, " so we accept H0: the distribution is NOT different between PA and", device, 'NDA hyperglicemia')    

<div class="alert alert-block alert-danger">
    
# **CLARKE ERROR GRID ANALYSIS**

</div>

In [ ]:
plt.rcdefaults()

dfw1 = pd.read_excel("Week 1.xlsx")
dfw2 = pd.read_excel("Week 2.xlsx")


dfw1['Datum'] = dfw1['Datum'].astype(str)
dfw2['Datum'] = dfw2['Datum'].astype(str)

dfw1.rename(columns={"Fingerprick  (mg/dL)": "Fingerprick (mg/dL)"}, inplace = True)

dfw1['Datum'] = dfw1['Datum'].str[11:13]
dfw2['Datum'] = dfw2['Datum'].str[11:13]
dfw1['Datum'] = dfw1['Datum'].astype(float)
dfw2['Datum'] = dfw2['Datum'].astype(float)

madrugadaw1 = dfw1.loc[(dfw1["Datum"] < 6)]
mañanaw1 = dfw1.loc[(dfw1["Datum"] >= 6) & (dfw1["Datum"] < 12)]
tardew1 = dfw1.loc[(dfw1["Datum"] >= 12) & (dfw1["Datum"] < 18)]
nochew1 = dfw1.loc[(dfw1["Datum"] >= 18)]

madrugadaw2 = dfw2.loc[(dfw2["Datum"] < 6)]
mañanaw2 = dfw2.loc[(dfw2["Datum"] >= 6) & (dfw2["Datum"] < 12)]
tardew2 = dfw2.loc[(dfw2["Datum"] >= 12) & (dfw2["Datum"] < 18)]
nochew2 = dfw2.loc[(dfw2["Datum"] >= 18)]

In [ ]:
import matplotlib.pyplot as plt

#This function takes in the reference values and the prediction values as lists and returns a list with each index corresponding to the total number
#of points within that zone (0=A, 1=B, 2=C, 3=D, 4=E) and the plot
def clarke_error_grid(ref_values, pred_values, device, period, title_string):
    #Checking to see if the lengths of the reference and prediction arrays are the same
    assert (len(ref_values) == len(pred_values)), "Unequal number of values (reference : {}) (prediction : {}).".format(len(ref_values), len(pred_values))
    #Checks to see if the values are within the normal physiological range, otherwise it gives a warning
    if max(ref_values) > 400 or max(pred_values) > 400:
        print("Input Warning: the maximum reference value {} or the maximum prediction value {} exceeds the normal physiological range of glucose (<400 mg/dl).".format(max(ref_values), max(pred_values)))
    if min(ref_values) < 0 or min(pred_values) < 0:
        print("Input Warning: the minimum reference value {} or the minimum prediction value {} is less than 0 mg/dl.".format(min(ref_values),  min(pred_values)))
    #Clear plot
    plt.clf()
    #Set up plot
    plt.scatter(ref_values, pred_values, marker='o', color='black', s=8)
    plt.title(title_string)
    plt.xlabel("Reference Concentration (mg/dl)")
    plt.ylabel("Prediction Concentration (mg/dl)")
    plt.xticks([0, 50, 100, 150, 200, 250, 300, 350, 400])
    plt.yticks([0, 50, 100, 150, 200, 250, 300, 350, 400])
    plt.gca().set_facecolor('white')
    #Set axes lengths
    plt.gca().set_xlim([0, 400])
    plt.gca().set_ylim([0, 400])
    plt.gca().set_aspect((400)/(400))
    #Plot zone lines
    plt.plot([0,400], [0,400], ':', c='black')                      #Theoretical 45 regression line
    plt.plot([0, 175/3], [70, 70], '-', c='black')
    #plt.plot([175/3, 320], [70, 400], '-', c='black')
    plt.plot([175/3, 400/1.2], [70, 400], '-', c='black')           #Replace 320 with 400/1.2 because 100*(400 - 400/1.2)/(400/1.2) =  20% error
    plt.plot([70, 70], [84, 400],'-', c='black')
    plt.plot([0, 70], [180, 180], '-', c='black')
    plt.plot([70, 290],[180, 400],'-', c='black')
    # plt.plot([70, 70], [0, 175/3], '-', c='black')
    plt.plot([70, 70], [0, 56], '-', c='black')                     #Replace 175.3 with 56 because 100*abs(56-70)/70) = 20% error
    # plt.plot([70, 400],[175/3, 320],'-', c='black')
    plt.plot([70, 400], [56, 320],'-', c='black')
    plt.plot([180, 180], [0, 70], '-', c='black')
    plt.plot([180, 400], [70, 70], '-', c='black')
    plt.plot([240, 240], [70, 180],'-', c='black')
    plt.plot([240, 400], [180, 180], '-', c='black')
    plt.plot([130, 180], [0, 70], '-', c='black')
    #Add zone titles
    plt.text(30, 15, "A", fontsize=15)
    plt.text(370, 260, "B", fontsize=15)
    plt.text(280, 370, "B", fontsize=15)
    plt.text(160, 370, "C", fontsize=15)
    plt.text(160, 15, "C", fontsize=15)
    plt.text(30, 140, "D", fontsize=15)
    plt.text(370, 120, "D", fontsize=15)
    plt.text(30, 370, "E", fontsize=15)
    plt.text(370, 15, "E", fontsize=15)
    #Statistics from the data
    zone = [0] * 5
    name = device+'CEGplot'+period+'.pdf'
    plt.savefig(name, bbox_inches = 'tight')
    
    for i in range(len(ref_values)):
        if (ref_values[i] <= 70 and pred_values[i] <= 70) or (pred_values[i] <= 1.2*ref_values[i] and pred_values[i] >= 0.8*ref_values[i]):
            zone[0] += 1    #Zone A
        elif (ref_values[i] >= 180 and pred_values[i] <= 70) or (ref_values[i] <= 70 and pred_values[i] >= 180):
            zone[4] += 1    #Zone E
        elif ((ref_values[i] >= 70 and ref_values[i] <= 290) and pred_values[i] >= ref_values[i] + 110) or ((ref_values[i] >= 130 and ref_values[i] <= 180) and (pred_values[i] <= (7/5)*ref_values[i] - 182)):
            zone[2] += 1    #Zone C
        elif (ref_values[i] >= 240 and (pred_values[i] >= 70 and pred_values[i] <= 180)) or (ref_values[i] <= 175/3 and pred_values[i] <= 180 and pred_values[i] >= 70) or ((ref_values[i] >= 175/3 and ref_values[i] <= 70) and pred_values[i] >= (6/5)*ref_values[i]):
            zone[3] += 1    #Zone D
        else:
            zone[1] += 1    #Zone B
            
    print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zone[0], zone[0]/sum(zone), zone[1], zone[1]/sum(zone), zone[2], zone[2]/sum(zone), zone[3], zone[3]/sum(zone), zone[4], zone[4]/sum(zone)))


    return plt, zone

In [ ]:
def two_proportion_test(len1, prop1, len2, prop2 , message):
    
    # p1 = x1/n1
    # p2 = x2/n2
    # H0: p1=p2 vs H1:p1 != p2
    n1 = len1
    n2 = len2
    n = n1+n2
    p1 = prop1
    p2 = prop2
    x1 = p1*n1
    x2 = p2*n2
    
    p_pool = (x1+x2)/n
    se = np.sqrt(p_pool*(1-p_pool)*(1/n1+1/n2))

    z_statistic = (p1-p2)/se
    p_value = 2*stats.norm().cdf(-1*np.abs(z_statistic))
    print('P-value for the 2 proportion test = ', p_value)

    if p_value < 0.05: 
        print('Thus, we reject the null hypothesis ({})'.format(message))
    else:
        print('Thus, we FAIL to reject the null hypothesis (no evidence that {})'.format(message))

In [ ]:
plt.rc('font', size = 12)

In [ ]:
plot, zonefslw1 = clarke_error_grid(dfw1["Fingerprick (mg/dL)"], dfw1["FSL (mg/dL)"], "FSL", "PA", "") #, "Clarke Error Grid -  Fingerprick vs FSL (Exercise)"
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zonefslw1[0], zonefslw1[0]/sum(zonefslw1), zonefslw1[1], zonefslw1[1]/sum(zonefslw1), zonefslw1[2], zonefslw1[2]/sum(zonefslw1), zonefslw1[3], zonefslw1[3]/sum(zonefslw1), zonefslw1[4], zonefslw1[4]/sum(zonefslw1)))

In [ ]:
plot, zonefslw2 = clarke_error_grid(dfw2["Fingerprick (mg/dL)"], dfw2["FSL (mg/dL)"],"FSL", "NDA", "") #, "Clarke Error Grid -  Fingerprick vs FSL (NDA)"
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zonefslw2[0], zonefslw2[0]/sum(zonefslw2), zonefslw2[1], zonefslw2[1]/sum(zonefslw2), zonefslw2[2], zonefslw2[2]/sum(zonefslw2), zonefslw2[3], zonefslw2[3]/sum(zonefslw2), zonefslw2[4], zonefslw2[4]/sum(zonefslw2)))

In [ ]:
two_proportion_test(zonefslw1[0], zonefslw1[0]/sum(zonefslw1), zonefslw2[0], zonefslw2[0]/sum(zonefslw2), "samples differ")

In [ ]:
plot, zoneesw1 = clarke_error_grid(dfw1["Fingerprick (mg/dL)"], dfw1["Eversense (mg/dl)"],"ES", "PA", "") #, "Clarke Error Grid -  Fingerprick vs Eversense (NDA)"
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zoneesw2[0], zoneesw2[0]/sum(zoneesw2), zoneesw2[1], zoneesw2[1]/sum(zoneesw2), zoneesw2[2], zoneesw2[2]/sum(zoneesw2), zoneesw2[3], zoneesw2[3]/sum(zoneesw2), zoneesw2[4], zoneesw2[4]/sum(zoneesw2)))

In [ ]:
plot, zoneesw2 = clarke_error_grid(dfw2["Fingerprick (mg/dL)"], dfw2["Eversense (mg/dl)"],"ES", "NDA", "") #, "Clarke Error Grid -  Fingerprick vs Eversense (NDA)"
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zoneesw2[0], zoneesw2[0]/sum(zoneesw2), zoneesw2[1], zoneesw2[1]/sum(zoneesw2), zoneesw2[2], zoneesw2[2]/sum(zoneesw2), zoneesw2[3], zoneesw2[3]/sum(zoneesw2), zoneesw2[4], zoneesw2[4]/sum(zoneesw2)))

In [ ]:
two_proportion_test(zoneesw1[0], zoneesw1[0]/sum(zoneesw1), zoneesw2[0], zoneesw2[0]/sum(zoneesw2), "samples differ")

<div class="alert alert-block alert-danger">
    
# **CLARKE ERROR GRID ANALYSIS -- time**

</div>

<div class="alert alert-block alert-warning ">
    
## **CLARKE ERROR GRID ANALYSIS (Morning)**

</div>

In [ ]:
plot, zonefslw1 = clarke_error_grid(list(mañanaw1["Fingerprick (mg/dL)"]), list(mañanaw1["FSL (mg/dL)"]),"FSL", "PA", "") #"Clarke Error Grid -  Fingerprick vs FSL (Morning - Exercise)"
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zonefslw1[0], zonefslw1[0]/sum(zonefslw1), zonefslw1[1], zonefslw1[1]/sum(zonefslw1), zonefslw1[2], zonefslw1[2]/sum(zonefslw1), zonefslw1[3], zonefslw1[3]/sum(zonefslw1), zonefslw1[4], zonefslw1[4]/sum(zonefslw1)))

In [ ]:
plot, zonefslw2 = clarke_error_grid(list(mañanaw2["Fingerprick (mg/dL)"]), list(mañanaw2["FSL (mg/dL)"]),"FSL", "NDA", "") #"Clarke Error Grid -  Fingerprick vs FSL (Morning - NDA)"
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zonefslw2[0], zonefslw2[0]/sum(zonefslw2), zonefslw2[1], zonefslw2[1]/sum(zonefslw2), zonefslw2[2], zonefslw2[2]/sum(zonefslw2), zonefslw2[3], zonefslw2[3]/sum(zonefslw2), zonefslw2[4], zonefslw2[4]/sum(zonefslw2)))

In [ ]:
two_proportion_test(zonefslw1[0], zonefslw1[0]/sum(zonefslw1), zonefslw2[0], zonefslw2[0]/sum(zonefslw2), "samples differ")

In [ ]:
plot, zoneesw1 = clarke_error_grid(list(mañanaw1["Fingerprick (mg/dL)"]), list(mañanaw1["Eversense (mg/dl)"]),"ES", "PA", "") #"Clarke Error Grid -  Fingerprick vs Eversense (Morning - Exercise)"
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zoneesw1[0], zoneesw1[0]/sum(zoneesw1), zoneesw1[1], zoneesw1[1]/sum(zoneesw1), zoneesw1[2], zoneesw1[2]/sum(zoneesw1), zoneesw1[3], zoneesw1[3]/sum(zoneesw1), zoneesw1[4], zoneesw1[4]/sum(zoneesw1)))

In [ ]:
plot, zoneesw2 = clarke_error_grid(list(mañanaw2["Fingerprick (mg/dL)"]), list(mañanaw2["Eversense (mg/dl)"]),"ES", "NDA", "") #"Clarke Error Grid -  Fingerprick vs Eversense (Morning - NDA)"
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zoneesw2[0], zoneesw2[0]/sum(zoneesw2), zoneesw2[1], zoneesw2[1]/sum(zoneesw2), zoneesw2[2], zoneesw2[2]/sum(zoneesw2), zoneesw2[3], zoneesw2[3]/sum(zoneesw2), zoneesw2[4], zoneesw2[4]/sum(zoneesw2)))

In [ ]:
two_proportion_test(zoneesw1[0], zoneesw1[0]/sum(zoneesw1), zoneesw2[0], zoneesw2[0]/sum(zoneesw2), "samples differ")

<div class="alert alert-block alert-warning ">
    
## **CLARKE ERROR GRID ANALYSIS (Aft)**

</div>

In [ ]:
plot, zonefslw1 = clarke_error_grid(list(tardew1["Fingerprick (mg/dL)"]), list(tardew1["FSL (mg/dL)"]),"FSL", "PA","")#, "Clarke Error Grid -  Fingerprick vs FSL (Night - NDA)"
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zonefslw1[0], zonefslw1[0]/sum(zonefslw1), zonefslw1[1], zonefslw1[1]/sum(zonefslw1), zonefslw1[2], zonefslw1[2]/sum(zonefslw1), zonefslw1[3], zonefslw1[3]/sum(zonefslw1), zonefslw1[4], zonefslw1[4]/sum(zonefslw1)))

In [ ]:
plot, zonefslw2 = clarke_error_grid(list(tardew2["Fingerprick (mg/dL)"]), list(tardew2["FSL (mg/dL)"]),"FSL", "NDA","")#, "Clarke Error Grid -  Fingerprick vs FSL (Afternoon - NDA)"
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zonefslw2[0], zonefslw2[0]/sum(zonefslw2), zonefslw2[1], zonefslw2[1]/sum(zonefslw2), zonefslw2[2], zonefslw2[2]/sum(zonefslw2), zonefslw2[3], zonefslw2[3]/sum(zonefslw2), zonefslw2[4], zonefslw2[4]/sum(zonefslw2)))

In [ ]:
two_proportion_test(zonefslw1[0], zonefslw1[0]/sum(zonefslw1), zonefslw2[0], zonefslw2[0]/sum(zonefslw2), "samples differ")

In [ ]:
plot, zoneesw1 = clarke_error_grid(list(tardew1["Fingerprick (mg/dL)"]), list(tardew1["Eversense (mg/dl)"]),"ES", "PA", "") #Clarke Error Grid -  Fingerprick vs Eversense (Afternoon - Exercise)
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zoneesw1[0], zoneesw1[0]/sum(zoneesw1), zoneesw1[1], zoneesw1[1]/sum(zoneesw1), zoneesw1[2], zoneesw1[2]/sum(zoneesw1), zoneesw1[3], zoneesw1[3]/sum(zoneesw1), zoneesw1[4], zoneesw1[4]/sum(zoneesw1)))

In [ ]:
plot, zoneesw2 = clarke_error_grid(list(tardew2["Fingerprick (mg/dL)"]), list(tardew2["Eversense (mg/dl)"]),"ES", "NDA", "") #Clarke Error Grid -  Fingerprick vs Eversense (Afternoon - NDA)
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zoneesw2[0], zoneesw2[0]/sum(zoneesw2), zoneesw2[1], zoneesw2[1]/sum(zoneesw2), zoneesw2[2], zoneesw2[2]/sum(zoneesw2), zoneesw2[3], zoneesw2[3]/sum(zoneesw2), zoneesw2[4], zoneesw2[4]/sum(zoneesw2)))

In [ ]:
two_proportion_test(zoneesw1[0], zoneesw1[0]/sum(zoneesw1), zoneesw2[0], zoneesw2[0]/sum(zoneesw2), "samples differ")

<div class="alert alert-block alert-warning ">
    
## **CLARKE ERROR GRID ANALYSIS (Night)**

</div>

In [ ]:
plot, zonefslw1 = clarke_error_grid(list(nochew1["Fingerprick (mg/dL)"]), list(nochew1["FSL (mg/dL)"]),"FSL", "PA", "") #Clarke Error Grid -  Fingerprick vs FSL (Night - exercise)
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zonefslw1[0], zonefslw1[0]/sum(zonefslw1), zonefslw1[1], zonefslw1[1]/sum(zonefslw1), zonefslw1[2], zonefslw1[2]/sum(zonefslw1), zonefslw1[3], zonefslw1[3]/sum(zonefslw1), zonefslw1[4], zonefslw1[4]/sum(zonefslw1)))

In [ ]:
plot, zonefslw2 = clarke_error_grid(list(nochew2["Fingerprick (mg/dL)"]), list(nochew2["FSL (mg/dL)"]),"FSL", "NDA", "") #Clarke Error Grid -  Fingerprick vs FSL (Night - NDA)
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zonefslw2[0], zonefslw2[0]/sum(zonefslw2), zonefslw2[1], zonefslw2[1]/sum(zonefslw2), zonefslw2[2], zonefslw2[2]/sum(zonefslw2), zonefslw2[3], zonefslw2[3]/sum(zonefslw2), zonefslw2[4], zonefslw2[4]/sum(zonefslw2)))

In [ ]:
two_proportion_test(zonefslw1[0], zonefslw1[0]/sum(zonefslw1), zonefslw2[0], zonefslw2[0]/sum(zonefslw2), "samples differ")

In [ ]:
plot, zoneesw1 = clarke_error_grid(list(nochew1["Fingerprick (mg/dL)"]), list(nochew1["Eversense (mg/dl)"]),"ES", "PA", "") #Clarke Error Grid -  Fingerprick vs Eversense (Night - Exercise)
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zoneesw1[0], zoneesw1[0]/sum(zoneesw1), zoneesw1[1], zoneesw1[1]/sum(zoneesw1), zoneesw1[2], zoneesw1[2]/sum(zoneesw1), zoneesw1[3], zoneesw1[3]/sum(zoneesw1), zoneesw1[4], zoneesw1[4]/sum(zoneesw1)))

In [ ]:
plot, zoneesw2 = clarke_error_grid(list(nochew2["Fingerprick (mg/dL)"]), list(nochew2["Eversense (mg/dl)"]),"ES", "NDA", "") #Clarke Error Grid -  Fingerprick vs Eversense (Night - list()NDA)
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zoneesw2[0], zoneesw2[0]/sum(zoneesw2), zoneesw2[1], zoneesw2[1]/sum(zoneesw2), zoneesw2[2], zoneesw2[2]/sum(zoneesw2), zoneesw2[3], zoneesw2[3]/sum(zoneesw2), zoneesw2[4], zoneesw2[4]/sum(zoneesw2)))

In [ ]:
two_proportion_test(zoneesw1[0], zoneesw1[0]/sum(zoneesw1), zoneesw2[0], zoneesw2[0]/sum(zoneesw2), "samples differ")

<div class="alert alert-block alert-warning ">
    
## **CLARKE ERROR GRID ANALYSIS (dawn)**

</div>

In [ ]:
plot, zonefslw1 = clarke_error_grid(list(madrugadaw1["Fingerprick (mg/dL)"]), list(madrugadaw1["FSL (mg/dL)"]),"FSL", "PA", "") #Clarke Error Grid -  Fingerprick vs FSL (Dawn - EXERCISE)
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zonefslw1[0], zonefslw1[0]/sum(zonefslw1), zonefslw1[1], zonefslw1[1]/sum(zonefslw1), zonefslw1[2], zonefslw1[2]/sum(zonefslw1), zonefslw1[3], zonefslw1[3]/sum(zonefslw1), zonefslw1[4], zonefslw1[4]/sum(zonefslw1)))

In [ ]:
plot, zonefslw2 = clarke_error_grid(list(madrugadaw2["Fingerprick (mg/dL)"]), list(madrugadaw2["FSL (mg/dL)"]),"FSL", "NDA", "") #Clarke Error Grid -  Fingerprick vs FSL (Dawn - NDA)
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zonefslw2[0], zonefslw2[0]/sum(zonefslw2), zonefslw2[1], zonefslw2[1]/sum(zonefslw2), zonefslw2[2], zonefslw2[2]/sum(zonefslw2), zonefslw2[3], zonefslw2[3]/sum(zonefslw2), zonefslw2[4], zonefslw2[4]/sum(zonefslw2)))

In [ ]:
two_proportion_test(zonefslw1[0], zonefslw1[0]/sum(zonefslw1), zonefslw2[0], zonefslw2[0]/sum(zonefslw2), "samples differ")

In [ ]:
plot, zoneesw1 = clarke_error_grid(list(madrugadaw1["Fingerprick (mg/dL)"]), list(madrugadaw1["Eversense (mg/dl)"]),"ES", "PA", "") #Clarke Error Grid -  Fingerprick vs Eversense (Dawn - Exercise)
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zoneesw1[0], zoneesw1[0]/sum(zoneesw1), zoneesw1[1], zoneesw1[1]/sum(zoneesw1), zoneesw1[2], zoneesw1[2]/sum(zoneesw1), zoneesw1[3], zoneesw1[3]/sum(zoneesw1), zoneesw1[4], zoneesw1[4]/sum(zoneesw1)))

In [ ]:
plot, zoneesw2 = clarke_error_grid(list(madrugadaw2["Fingerprick (mg/dL)"]), list(madrugadaw2["Eversense (mg/dl)"]),"FSL", "NDA", "") #Clarke Error Grid -  Fingerprick vs Eversense (Dawn - list(NDA)
print("There are: \n {0} values in zone A ({1}%), \n {2} values in zone B ({3}%), \n {4} values in zone C ({5}%), \n {6} values in zone D ({7}%), \n {8} values in zone E ({9}%)".format(zoneesw2[0], zoneesw2[0]/sum(zoneesw2), zoneesw2[1], zoneesw2[1]/sum(zoneesw2), zoneesw2[2], zoneesw2[2]/sum(zoneesw2), zoneesw2[3], zoneesw2[3]/sum(zoneesw2), zoneesw2[4], zoneesw2[4]/sum(zoneesw2)))

In [ ]:
two_proportion_test(zoneesw1[0], zoneesw1[0]/sum(zoneesw1), zoneesw2[0], zoneesw2[0]/sum(zoneesw2), "samples differ")

<div class="alert alert-block alert-warning">
    
# **CLARKE ERROR GRID ANALYSIS -- glucose**

</div>

In [ ]:
# FSL
w170_fsl = dfw1[dfw1["FSL (mg/dL)"] <= 70]
w270_fsl = dfw2[dfw2["FSL (mg/dL)"] <= 70]
w1180_fsl = dfw1.loc[(dfw1["FSL (mg/dL)"] <= 180) & (dfw1["FSL (mg/dL)"] > 70)]
w2180_fsl = dfw2.loc[(dfw2["FSL (mg/dL)"] <= 180) & (dfw2["FSL (mg/dL)"] > 70)]
w1max_fsl = dfw1[dfw1["FSL (mg/dL)"] > 180]
w2max_fsl = dfw2[dfw2["FSL (mg/dL)"] > 180]

#Eversense
w170_es = dfw1[dfw1["Eversense (mg/dl)"] <= 70]
w270_es = dfw2[dfw2["Eversense (mg/dl)"] <= 70]
w1180_es = dfw1.loc[(dfw1["Eversense (mg/dl)"] <= 180) & (dfw1["Eversense (mg/dl)"] > 70)]
w2180_es = dfw2.loc[(dfw2["Eversense (mg/dl)"] <= 180) & (dfw2["Eversense (mg/dl)"] > 70)]
w1max_es = dfw1[dfw1["Eversense (mg/dl)"] > 180]
w2max_es = dfw2[dfw2["Eversense (mg/dl)"] > 180]

#Fingerprick
w170_fp = dfw1[dfw1["Fingerprick (mg/dL)"] <= 70]
w270_fp = dfw2[dfw2["Fingerprick (mg/dL)"] <= 70]
w1180_fp = dfw1.loc[(dfw1["Fingerprick (mg/dL)"] <= 180) & (dfw1["Fingerprick (mg/dL)"] > 70)]
w2180_fp = dfw2.loc[(dfw2["Fingerprick (mg/dL)"] <= 180) & (dfw2["Fingerprick (mg/dL)"] > 70)]
w1max_fp = dfw1[dfw1["Fingerprick (mg/dL)"] > 180]
w2max_fp = dfw2[dfw2["Fingerprick (mg/dL)"] > 180]

In [ ]:
### FSL
plot, zonefslw1hypo = clarke_error_grid(list(w170_fp["Fingerprick (mg/dL)"]), list(w170_fp["FSL (mg/dL)"]),"FSL", "PA", "");
plot, zonefslw2hypo = clarke_error_grid(list(w270_fp["Fingerprick (mg/dL)"]), list(w270_fp["FSL (mg/dL)"]),"FSL", "NDA", "");

In [ ]:
two_proportion_test(zonefslw1hypo[0], zonefslw1hypo[0]/sum(zonefslw1hypo), zonefslw2hypo[0], zonefslw2hypo[0]/sum(zonefslw2hypo), "samples differ")

In [ ]:
plot, zonefslw1normo = clarke_error_grid(list(w1180_fp["Fingerprick (mg/dL)"]),list( w1180_fp["FSL (mg/dL)"]),"FSL", "PA", "")
plot, zonefslw2normo = clarke_error_grid(list(w2180_fp["Fingerprick (mg/dL)"]),list( w2180_fp["FSL (mg/dL)"]),"FSL", "NDA", "")

In [ ]:
two_proportion_test(zonefslw1normo[0], zonefslw1normo[0]/sum(zonefslw1normo), zonefslw2normo[0], zonefslw2normo[0]/sum(zonefslw2normo), "samples differ")

In [ ]:
plot, zonefslw1hyper = clarke_error_grid(list(w1max_fp["Fingerprick (mg/dL)"]),list( w1max_fp["FSL (mg/dL)"]),"FSL", "PA", "")
plot, zonefslw2hyper = clarke_error_grid(list(w2max_fp["Fingerprick (mg/dL)"]),list( w2max_fp["FSL (mg/dL)"]),"FSL", "NDA", "")

In [ ]:
two_proportion_test(zonefslw1hyper[0], zonefslw1hyper[0]/sum(zonefslw1hyper), zonefslw2hyper[0], zonefslw2hyper[0]/sum(zonefslw2hyper), "samples differ")

In [ ]:
#### EVERSENSE
plot, zoneesw1hypo = clarke_error_grid(list(w170_fp["Fingerprick (mg/dL)"]), list(w170_fp["Eversense (mg/dl)"]),"ES", "PA", "")
plot, zoneesw2hypo = clarke_error_grid(list(w270_fp["Fingerprick (mg/dL)"]), list(w270_fp["Eversense (mg/dl)"]),"ES", "NDA", "")

In [ ]:
two_proportion_test(zoneesw1hypo[0], zoneesw1hypo[0]/sum(zoneesw1hypo), zoneesw2hypo[0], zoneesw2hypo[0]/sum(zoneesw2hypo), "samples differ")

In [ ]:
plot, zoneesw1normo = clarke_error_grid(list(w1180_fp["Fingerprick (mg/dL)"]),list( w1180_fp["Eversense (mg/dl)"]),"ES", "PA", "")
plot, zoneesw2normo = clarke_error_grid(list(w2180_fp["Fingerprick (mg/dL)"]),list( w2180_fp["Eversense (mg/dl)"]),"ES", "NDA", "")

In [ ]:
two_proportion_test(zoneesw1normo[0], zoneesw1normo[0]/sum(zoneesw1normo), zoneesw2normo[0], zoneesw2normo[0]/sum(zoneesw2normo), "samples differ")

In [ ]:
plot, zoneesw1hyper = clarke_error_grid(list(w1max_fp["Fingerprick (mg/dL)"]),list( w1max_fp["Eversense (mg/dl)"]),"ES", "PA", "")
plot, zoneesw2hyper = clarke_error_grid(list(w2max_fp["Fingerprick (mg/dL)"]),list( w2max_fp["Eversense (mg/dl)"]),"ES", "NDA", "")

In [ ]:
two_proportion_test(zoneesw1hyper[0], zoneesw1hyper[0]/sum(zoneesw1hyper), zoneesw2hyper[0], zoneesw2hyper[0]/sum(zoneesw2hyper), "samples differ")

<div class="alert alert-block alert-danger">
    
# **PA vs NDA**

</div>

<div class="alert alert-block alert-warning">
    
**PA vs NDA -- time**

</div>

In [ ]:
madrugadaw1 = dfw1.loc[(dfw1["Datum"] < 6)]
mañanaw1 = dfw1.loc[(dfw1["Datum"] >= 6) & (dfw1["Datum"] < 12)]
tardew1 = dfw1.loc[(dfw1["Datum"] >= 12) & (dfw1["Datum"] < 18)]
nochew1 = dfw1.loc[(dfw1["Datum"] >= 18)]

madrugadaw2 = dfw2.loc[(dfw2["Datum"] < 6)]
mañanaw2 = dfw2.loc[(dfw2["Datum"] >= 6) & (dfw2["Datum"] < 12)]
tardew2 = dfw2.loc[(dfw2["Datum"] >= 12) & (dfw2["Datum"] < 18)]
nochew2 = dfw2.loc[(dfw2["Datum"] >= 18)]

In [ ]:
np.random.seed(7)
import random
random.seed(7)

def bootstrap(pop1, pop2, alpha, periodo1, periodo2, week, alternatives):
    n = len(pop1)
    m = len(pop2)
    random.seed(7)
    for e in alternatives:
        
        populationmean = []
        populationmedian = []

        for i in range(0,1000):
            resampledpop1 = choices(pop1[e].values, k = n)
            resampledpop2 = choices(pop2[e].values, k = m)
            populationmean.append(np.mean(resampledpop1) - np.mean(resampledpop2))
            populationmedian.append(np.median(resampledpop1) - np.median(resampledpop2))
        ci_mean = stats.t.interval(alpha=0.95, df=len(populationmean)-1,loc=np.mean(populationmean),scale=stats.sem(populationmean))
        lowermean = ci_mean[0]
        uppermean = ci_mean[1]
        ci_median = stats.t.interval(alpha=0.95, df=len(populationmedian)-1,loc=np.mean(populationmedian),scale=stats.sem(populationmedian))
        lowermedian =ci_median[0]
        uppermedian =ci_median[1]
        print('median' ,lowermedian, uppermedian)
    return [lowermean, uppermean], [lowermedian, uppermedian]

In [ ]:
intervalo = 'Morning'
tval, pval = stats.mannwhitneyu(mañanaw1['Fingerprick (mg/dL)'], mañanaw2['Fingerprick (mg/dL)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", intervalo, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", intervalo, "distribution is NOT different between Week1 and Week2")
    
intervalo = 'Afternoon'
tval, pval = stats.mannwhitneyu(tardew1['Fingerprick (mg/dL)'], tardew2['Fingerprick (mg/dL)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", intervalo, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", intervalo, "distribution is NOT different between Week1 and Week2")
    
    
intervalo = 'Evening'
tval, pval = stats.mannwhitneyu(nochew1['Fingerprick (mg/dL)'], nochew2['Fingerprick (mg/dL)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", intervalo, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", intervalo, "distribution is NOT different between Week1 and Week2")
    
    
intervalo = 'Night'
tval, pval = stats.mannwhitneyu(madrugadaw1['Fingerprick (mg/dL)'], madrugadaw2['Fingerprick (mg/dL)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", intervalo, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", intervalo, "distribution is NOT different between Week1 and Week2")

In [ ]:
means_m, medians_m = bootstrap(mañanaw1, mañanaw2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,["Fingerprick (mg/dL)"])
means_a, medians_a = bootstrap(tardew1, tardew2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,["Fingerprick (mg/dL)"])
means_e, medians_e = bootstrap(nochew1, nochew2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,["Fingerprick (mg/dL)"])
means_n, medians_n = bootstrap(madrugadaw1, madrugadaw2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,["Fingerprick (mg/dL)"])

dfci_medians = pd.DataFrame() 
dfci_medians['var_name'] = ['$M_{int}$', '$A_{int}$', '$E_{int}$', '$N_{int}$']
dfci_medians['varixable_name'] = ['$M_{int}$', '$A_{int}$', '$E_{int}$', '$N_{int}$']
dfci_medians['ci_upper'] = [medians_m[1],medians_e[1],medians_e[1],medians_n[1]]
dfci_medians['ci_lower'] = [medians_m[0],medians_e[0],medians_e[0],medians_n[0]]
dfci_medians['median'] = [np.mean(medians_m),np.mean(medians_a),np.mean(medians_e),np.mean(medians_n)]
# plot_custom_boxplot_with_ci(dfci_medians,'mediana_t_fp', type_variables='', flag_save_figure=True,flag_save_features=False)

In [ ]:
intervalo = 'Morning'
tval, pval = stats.mannwhitneyu(mañanaw1['FSL (mg/dL)'], mañanaw2['FSL (mg/dL)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", intervalo, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", intervalo, "distribution is NOT different between Week1 and Week2")
    
intervalo = 'Afternoon'
tval, pval = stats.mannwhitneyu(tardew1['FSL (mg/dL)'], tardew2['FSL (mg/dL)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", intervalo, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", intervalo, "distribution is NOT different between Week1 and Week2")
    
    
intervalo = 'Evening'
tval, pval = stats.mannwhitneyu(nochew1['FSL (mg/dL)'], nochew2['FSL (mg/dL)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", intervalo, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", intervalo, "distribution is NOT different between Week1 and Week2")
    
    
intervalo = 'Night'
tval, pval = stats.mannwhitneyu(madrugadaw1['FSL (mg/dL)'], madrugadaw2['FSL (mg/dL)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", intervalo, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", intervalo, "distribution is NOT different between Week1 and Week2")

In [ ]:
means_m, medians_m = bootstrap(mañanaw1, mañanaw2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,["FSL (mg/dL)"])
means_a, medians_a = bootstrap(tardew1, tardew2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,["FSL (mg/dL)"])
means_e, medians_e = bootstrap(nochew1, nochew2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,["FSL (mg/dL)"])
means_n, medians_n = bootstrap(madrugadaw1, madrugadaw2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,["FSL (mg/dL)"])

dfci_medians = pd.DataFrame()
dfci_medians['var_name'] = ['$M_{int}$', '$A_{int}$', '$E_{int}$', '$N_{int}$']
dfci_medians['variable_name'] = ['$M_{int}$', '$A_{int}$', '$E_{int}$', '$N_{int}$']
dfci_medians['ci_upper'] = [medians_m[1],medians_e[1],medians_e[1],medians_n[1]]
dfci_medians['ci_lower'] = [medians_m[0],medians_e[0],medians_e[0],medians_n[0]]
dfci_medians['median'] = [np.mean(medians_m),np.mean(medians_a),np.mean(medians_e),np.mean(medians_n)]
# plot_custom_boxplot_with_ci(dfci_medians,'mediana_t_fsl',type_variables='', flag_save_figure=True,
#                             flag_save_features=False)

In [ ]:
intervalo = 'Morning'
tval, pval = stats.mannwhitneyu(mañanaw1['Eversense (mg/dl)'], mañanaw2['Eversense (mg/dl)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", intervalo, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", intervalo, "distribution is NOT different between Week1 and Week2")
    
intervalo = 'Afternoon'
tval, pval = stats.mannwhitneyu(tardew1['Eversense (mg/dl)'], tardew2['Eversense (mg/dl)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", intervalo, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", intervalo, "distribution is NOT different between Week1 and Week2")
    
    
intervalo = 'Evening'
tval, pval = stats.mannwhitneyu(nochew1['Eversense (mg/dl)'], nochew2['Eversense (mg/dl)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", intervalo, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", intervalo, "distribution is NOT different between Week1 and Week2")
    
    
intervalo = 'Night'
tval, pval = stats.mannwhitneyu(madrugadaw1['Eversense (mg/dl)'], madrugadaw2['Eversense (mg/dl)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", intervalo, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", intervalo, "distribution is NOT different between Week1 and Week2")

In [ ]:
means_m, medians_m = bootstrap(mañanaw1, mañanaw2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,["Eversense (mg/dl)"])
means_a, medians_a = bootstrap(tardew1, tardew2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,["Eversense (mg/dl)"])
means_e, medians_e = bootstrap(nochew1, nochew2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,["Eversense (mg/dl)"])
means_n, medians_n = bootstrap(madrugadaw1, madrugadaw2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,["Eversense (mg/dl)"])

dfci_medians = pd.DataFrame()
dfci_medians['var_name'] = ['$M_{int}$', '$A_{int}$', '$E_{int}$', '$N_{int}$']
dfci_medians['variable_name'] = ['$M_{int}$', '$A_{int}$', '$E_{int}$', '$N_{int}$']
dfci_medians['ci_upper'] = [medians_m[1],medians_e[1],medians_e[1],medians_n[1]]
dfci_medians['ci_lower'] = [medians_m[0],medians_e[0],medians_e[0],medians_n[0]]
dfci_medians['median'] = [np.mean(medians_m),np.mean(medians_a),np.mean(medians_e),np.mean(medians_n)]
# plot_custom_boxplot_with_ci(dfci_medians,'mediana_t_Es', type_variables='', flag_save_figure=True,
#                             flag_save_features=False)

<div class="alert alert-block alert-warning">
    
**PA vs NDA -- glucose**

</div>

In [ ]:
#PA
hypo_ref_w1 = dfw1.loc[(dfw1['Fingerprick (mg/dL)'] < 70)]
normo_ref_w1 = dfw1.loc[(dfw1['Fingerprick (mg/dL)'] >= 70) & (dfw1['Fingerprick (mg/dL)'] < 180)]
hyper_ref_w1 = dfw1.loc[(dfw1['Fingerprick (mg/dL)'] >= 180)]

#NDA
hypo_ref_w2 = dfw2.loc[(dfw2['Fingerprick (mg/dL)'] < 70)]
normo_ref_w2 = dfw2.loc[(dfw2['Fingerprick (mg/dL)'] >= 70) & (dfw2['Fingerprick (mg/dL)'] < 180)]
hyper_ref_w2 = dfw2.loc[(dfw2['Fingerprick (mg/dL)'] >= 180)]

rango = 'Hypo'
tval, pval = stats.mannwhitneyu(hypo_ref_w1['Fingerprick (mg/dL)'], hypo_ref_w2['Fingerprick (mg/dL)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", rango, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", rango, "distribution is NOT different between Week1 and Week2")
    
rango = 'Normo'
tval, pval = stats.mannwhitneyu(normo_ref_w1['Fingerprick (mg/dL)'], normo_ref_w2['Fingerprick (mg/dL)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", rango, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", rango, "distribution is NOT different between Week1 and Week2") 
    
rango = 'hyper'
tval, pval = stats.mannwhitneyu(hyper_ref_w1['Fingerprick (mg/dL)'], hyper_ref_w2['Fingerprick (mg/dL)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", rango, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", rango, "distribution is NOT different between Week1 and Week2")

In [ ]:
means_hypo, medians_hypo = bootstrap(hypo_ref_w1, hypo_ref_w2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,["Fingerprick (mg/dL)"])
means_hyperu, medians_hyperu = bootstrap(normo_ref_w1, normo_ref_w2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,["Fingerprick (mg/dL)"])
means_hyper, medians_hyper = bootstrap(hyper_ref_w1, hyper_ref_w2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,["Fingerprick (mg/dL)"])

dfci_hypoedians = pd.DataFrame()
dfci_hypoedians['var_name'] = ['Hypoglycemia', 'Euglycemia', 'Hyperglycemia']
dfci_hypoedians['variable_name'] = ['Hypoglycemia', 'Euglycemia', 'Hyperglycemia']
dfci_hypoedians['ci_upper'] = [medians_hypo[1],medians_hyper[1],medians_hyper[1]]
dfci_hypoedians['ci_lower'] = [medians_hypo[0],medians_hyper[0],medians_hyper[0]]
dfci_hypoedians['median'] = [np.mean(medians_hypo),np.mean(medians_hyperu),np.mean(medians_hyper)]
# plot_custom_boxplot_with_ci(dfci_hypoedians,'mediana_gluc_fp', type_variables='', flag_save_figure=True,
#                             flag_save_features=False)

In [ ]:
#PA
hypo_ref_w1 = dfw1.loc[(dfw1['FSL (mg/dL)'] < 70)]
normo_ref_w1 = dfw1.loc[(dfw1['FSL (mg/dL)'] >= 70) & (dfw1['FSL (mg/dL)'] < 180)]
hyper_ref_w1 = dfw1.loc[(dfw1['FSL (mg/dL)'] >= 180)]

#NDA
hypo_ref_w2 = dfw2.loc[(dfw2['FSL (mg/dL)'] < 70)]
normo_ref_w2 = dfw2.loc[(dfw2['FSL (mg/dL)'] >= 70) & (dfw2['FSL (mg/dL)'] < 180)]
hyper_ref_w2 = dfw2.loc[(dfw2['FSL (mg/dL)'] >= 180)]

rango = 'Hypo'
tval, pval = stats.mannwhitneyu(hypo_ref_w1['FSL (mg/dL)'], hypo_ref_w2['FSL (mg/dL)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", rango, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", rango, "distribution is NOT different between Week1 and Week2")
    
rango = 'Normo'
tval, pval = stats.mannwhitneyu(normo_ref_w1['FSL (mg/dL)'], normo_ref_w2['FSL (mg/dL)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", rango, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", rango, "distribution is NOT different between Week1 and Week2")
    
    
rango = 'hyper'
tval, pval = stats.mannwhitneyu(hyper_ref_w1['FSL (mg/dL)'], hyper_ref_w2['FSL (mg/dL)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", rango, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", rango, "distribution is NOT different between Week1 and Week2")

In [ ]:
means_hypo, medians_hypo = bootstrap(hypo_ref_w1, hypo_ref_w2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,['FSL (mg/dL)'])
means_hyperu, medians_hyperu = bootstrap(normo_ref_w1, normo_ref_w2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,['FSL (mg/dL)'])
means_hyper, medians_hyper = bootstrap(hyper_ref_w1, hyper_ref_w2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,['FSL (mg/dL)'])

dfci_hypoedians = pd.DataFrame()
dfci_hypoedians['var_name'] = ['Hypoglycemia', 'Euglycemia', 'Hyperglycemia']
dfci_hypoedians['variable_name'] = ['Hypoglycemia', 'Euglycemia', 'Hyperglycemia']
dfci_hypoedians['ci_upper'] = [medians_hypo[1],medians_hyper[1],medians_hyper[1]]
dfci_hypoedians['ci_lower'] = [medians_hypo[0],medians_hyper[0],medians_hyper[0]]
dfci_hypoedians['median'] = [np.mean(medians_hypo),np.mean(medians_hyperu),np.mean(medians_hyper)]
# plot_custom_boxplot_with_ci(dfci_hypoedians, 'mediana_gluc_fsl',type_variables='', flag_save_figure=True,
#                             flag_save_features=False)

In [ ]:
#PA
hypo_ref_w1 = dfw1.loc[(dfw1['Eversense (mg/dl)'] < 70)]
normo_ref_w1 = dfw1.loc[(dfw1['Eversense (mg/dl)'] >= 70) & (dfw1['Eversense (mg/dl)'] < 180)]
hyper_ref_w1 = dfw1.loc[(dfw1['Eversense (mg/dl)'] >= 180)]

#NDA
hypo_ref_w2 = dfw2.loc[(dfw2['Eversense (mg/dl)'] < 70)]
normo_ref_w2 = dfw2.loc[(dfw2['Eversense (mg/dl)'] >= 70) & (dfw2['Eversense (mg/dl)'] < 180)]
hyper_ref_w2 = dfw2.loc[(dfw2['Eversense (mg/dl)'] >= 180)]

rango = 'Hypo'
tval, pval = stats.mannwhitneyu(hypo_ref_w1['Eversense (mg/dl)'], hypo_ref_w2['Eversense (mg/dl)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", rango, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", rango, "distribution is NOT different between Week1 and Week2")
    
rango = 'Normo'
tval, pval = stats.mannwhitneyu(normo_ref_w1['Eversense (mg/dl)'], normo_ref_w2['Eversense (mg/dl)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", rango, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", rango, "distribution is NOT different between Week1 and Week2")
    
    
rango = 'hyper'
tval, pval = stats.mannwhitneyu(hyper_ref_w1['Eversense (mg/dl)'], hyper_ref_w2['Eversense (mg/dl)'])
if pval < 0.05:
    print("The pvalue is: ", pval, " so we reject H0: the", rango, "distribution is different between Week1 and Week2")
else:
    print("The pvalue is: ", pval, " so we accept H0: the ", rango, "distribution is NOT different between Week1 and Week2")

In [ ]:
means_hypo, medians_hypo = bootstrap(hypo_ref_w1, hypo_ref_w2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,['Eversense (mg/dl)'])
means_hyperu, medians_hyperu = bootstrap(normo_ref_w1, normo_ref_w2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,['Eversense (mg/dl)'])
means_hyper, medians_hyper = bootstrap(hyper_ref_w1, hyper_ref_w2, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,['Eversense (mg/dl)'])

dfci_hypoedians = pd.DataFrame()
dfci_hypoedians['var_name'] = ['Hypoglycemia', 'Euglycemia', 'Hyperglycemia']
dfci_hypoedians['variable_name'] = ['Hypoglycemia', 'Euglycemia', 'Hyperglycemia']
dfci_hypoedians['ci_upper'] = [medians_hypo[1],medians_hyper[1],medians_hyper[1]]
dfci_hypoedians['ci_lower'] = [medians_hypo[0],medians_hyper[0],medians_hyper[0]]
dfci_hypoedians['median'] = [np.mean(medians_hypo),np.mean(medians_hyperu),np.mean(medians_hyper)]
# plot_custom_boxplot_with_ci(dfci_hypoedians,'mediana_gluc_es', type_variables='', flag_save_figure=True,
#                             flag_save_features=False)

<div class="alert alert-block alert-danger">
    
# **Kruskal Wallis**

</div>

In [ ]:
dfw1 = pd.read_excel("Week 1.xlsx")
dfw2 = pd.read_excel("Week 2.xlsx")


dfw1['Datum'] = dfw1['Datum'].astype(str)
dfw2['Datum'] = dfw2['Datum'].astype(str)

dfw1.rename(columns={"Fingerprick  (mg/dL)": "Fingerprick (mg/dL)"}, inplace = True)

dfw1['Datum'] = dfw1['Datum'].str[11:13]
dfw2['Datum'] = dfw2['Datum'].str[11:13]
dfw1['Datum'] = dfw1['Datum'].astype(float)
dfw2['Datum'] = dfw2['Datum'].astype(float)

madrugadaw1 = dfw1.loc[(dfw1["Datum"] < 8)]
mañanaw1 = dfw1.loc[(dfw1["Datum"] >= 8) & (dfw1["Datum"] < 16)]
tardew1 = dfw1.loc[(dfw1["Datum"] >= 16)]

madrugadaw2 = dfw2.loc[(dfw2["Datum"] < 8)]
mañanaw2 = dfw2.loc[(dfw2["Datum"] >= 8) & (dfw2["Datum"] < 16)]
tardew2 = dfw2.loc[(dfw2["Datum"] >= 16)]

In [ ]:
madrugadaw1 = dfw1.loc[(dfw1["Datum"] < 6)]
mañanaw1 = dfw1.loc[(dfw1["Datum"] >= 6) & (dfw1["Datum"] < 12)]
tardew1 = dfw1.loc[(dfw1["Datum"] >= 12) & (dfw1["Datum"] < 18)]
nochew1 = dfw1.loc[(dfw1["Datum"] >= 18)]
fullw1 = dfw1

madrugadaw2 = dfw2.loc[(dfw2["Datum"] < 6)]
mañanaw2 = dfw2.loc[(dfw2["Datum"] >= 6) & (dfw2["Datum"] < 12)]
tardew2 = dfw2.loc[(dfw2["Datum"] >= 12) & (dfw2["Datum"] < 18)]
nochew2 = dfw2.loc[(dfw2["Datum"] >= 18)]
fullw2 = dfw2

In [ ]:
cont = 0
alternatives = ["Fingerprick (mg/dL)"]
for e in alternatives:
    plt.figure(figsize=(30,4))

    a = madrugadaw1[e].values
    b = mañanaw1[e].values
    c = tardew1[e].values
    d = nochew1[e].values
    _, pval = stats.kruskal(madrugadaw1[e].values, mañanaw1[e].values, tardew1[e].values, nochew1[e].values)

    if pval < 0.05:
        print("For ", e, "the p value is: ", np.round(pval,5), "so, we reject the null hypothesis, \n there are differences among the groups")
    else:
        print("For ", e, "the p value is:", np.round(pval,5), "so, we accept the null hypothesis, \n there are no differences among the groups")

    cont += 1
    plt.subplot(1,3,cont)
    plt.hist(a, alpha = 1, label = "Night PA")
    plt.hist(b, alpha = 0.20, label = "Morning PA")
    plt.hist(c, alpha = 0.20, label = "Afternoon PA")
    plt.hist(d, alpha = 0.20, label = "Evening PA")
    plt.legend()
    #plt.title(e + " Week 1")
    plt.show()

### Bootstrapping Week 1

In [ ]:
np.random.seed(7)
import random
random.seed(7)

def bootstrap(pop1, pop2, alpha, periodo1, periodo2, week, alternatives):
    n = len(pop1)
    m = len(pop2)
    random.seed(7)
    for e in alternatives:
        
        populationmean = []
        populationmedian = []

        for i in range(0,1000):
            resampledpop1 = choices(pop1[e].values, k = n)
            resampledpop2 = choices(pop2[e].values, k = m)
            populationmean.append(np.mean(resampledpop1) - np.mean(resampledpop2))
            populationmedian.append(np.median(resampledpop1) - np.median(resampledpop2))
        ci_mean = stats.t.interval(alpha=0.95, df=len(populationmean)-1,loc=np.mean(populationmean),scale=stats.sem(populationmean))
        lowermean = ci_mean[0]
        uppermean = ci_mean[1]
        ci_median = stats.t.interval(alpha=0.95, df=len(populationmedian)-1,loc=np.mean(populationmedian),scale=stats.sem(populationmedian))
        lowermedian =ci_median[0]
        uppermedian =ci_median[1]
        print('median' ,lowermedian, uppermedian)
    return [lowermean, uppermean], [lowermedian, uppermedian]

In [ ]:
means_nm, medians_nm  =bootstrap(madrugadaw1, mañanaw1, 0.05, "00:00 - 06:00", "06:00 - 12:00",1,["Fingerprick (mg/dL)"])

In [ ]:
means_na, medians_na = bootstrap(madrugadaw1, tardew1, 0.05, "00:00 - 06:00", "12:00 - 18:00",1,["Fingerprick (mg/dL)"])

In [ ]:
means_ne, medians_ne = bootstrap(madrugadaw1, nochew1, 0.05, "00:00 - 06:00", "18:00 - 23:59",1,["Fingerprick (mg/dL)"])

In [ ]:
means_ma, medians_ma = bootstrap(mañanaw1, tardew1, 0.05, "06:00 - 12:00", "12:00 - 18:00",1,["Fingerprick (mg/dL)"])

In [ ]:
means_me, medians_me = bootstrap(mañanaw1, nochew1, 0.05, "06:00 - 12:00", "18:00 - 23:59",1,["Fingerprick (mg/dL)"])

In [ ]:
means_ea, medians_ea = bootstrap(tardew1, nochew1, 0.05, "12:00 - 18:00", "18:00 - 23:59",1,["Fingerprick (mg/dL)"])

In [ ]:
cidf = pd.DataFrame()
cidf['variable_name'] = ['$N_{int}.M_{int}$', '$N_{int}.A_{int}$', '$N_{int}.E_{int}$', '$M_{int}.A_{int}$', '$M_{int}.E_{int}$','$A_{int}.E_{int}$']
cidf['var_name'] = ['$N_{int}.M_{int}', '$N_{int}.A_{int}', '$N_{int}.E_{int}', '$M_{int}.A_{int}', '$M_{int}.E_{int}','$A_{int}.E_{int}']
cidf['ci_upper'] = [medians_nm[1],medians_na[1],medians_ne[1],medians_ma[1],medians_me[1],medians_ea[1]]
cidf['ci_lower'] = [medians_nm[0],medians_na[0],medians_ne[0],medians_ma[0], medians_me[0], medians_ea[0]]
cidf['mean'] = [np.mean(medians_nm),np.mean(medians_na),np.mean(medians_ne),np.mean(medians_ma), np.mean(medians_me), np.mean(medians_ea)]

plot_custom_boxplot_with_ci(cidf, 'KW_partd_medians', type_variables='', flag_save_figure=True,
                            flag_save_features=False)

In [ ]:
figure9 = cidf[['variable_name', 'ci_upper', 'ci_lower', 'mean']]

In [ ]:
figure9.to_excel('figure_9.xlsx', index= False)

In [ ]:
cidf = pd.DataFrame()
cidf['variable_name'] = ['$N_{int}.M_{int}$', '$N_{int}.A_{int}$', '$N_{int}.E_{int}$', '$M_{int}.A_{int}$', '$M_{int}.E_{int}$','$A_{int}.E_{int}$']
cidf['var_name'] = ['$N_{int}.M_{int}', '$N_{int}.A_{int}', '$N_{int}.E_{int}', '$M_{int}.A_{int}', '$M_{int}.E_{int}','$A_{int}.E_{int}']
cidf['ci_upper'] = [means_nm[1],means_na[1],means_ne[1],means_ma[1],means_me[1],means_ea[1]]
cidf['ci_lower'] = [means_nm[0],means_nm[0],means_nm[0],means_nm[0], means_me[0], means_ea[0]]
cidf['mean'] = [np.mean(means_nm),np.mean(means_na),np.mean(means_ne),np.mean(means_ma), np.mean(means_me), np.mean(means_ea)]

plot_custom_boxplot_with_ci(cidf, 'KW_partd_means', type_variables='', flag_save_figure=True,
                            flag_save_features=False)